In [ ]:
#you'll need to pip install a few things (again this is in linux but it shouldn't be too hard to install these packages in mac)
#!pip install mido
#!pip install midi2audio
#!sudo apt-get install fluidsynth

In [23]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm, second2tick, tick2second
from midi2audio import FluidSynth
from IPython.display import Audio
import os

In [27]:
def generate_chords(start, end, chord_type='major', num_notes=3):
    if chord_type not in ['major', 'minor', 'dom7', 'diminished', 'augmented', 'sus2', 'sus4']:
        raise ValueError('Invalid chord type. Must be one of "major", "minor", "dom7", "diminished", "augmented", "sus2", "sus4".')

    chords = []
    for root in range(start, end+1):
        if chord_type == 'major':
            chord = [root, root + 4, root + 7]  # Major triad
            if num_notes >= 4:
                chord.append(root + 11)  # Major seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Major ninth
        elif chord_type == 'minor':
            chord = [root, root + 3, root + 7]  # Minor triad
            if num_notes >= 4:
                chord.append(root + 10)  # Minor seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Minor ninth
        elif chord_type == 'dom7':
            chord = [root, root + 4, root + 7, root + 10]  # Dominant 7th
            if num_notes >= 5:
                chord.append(root + 14)  # Dominant 9th
        elif chord_type == 'diminished':
            chord = [root, root + 3, root + 6]
        elif chord_type == 'augmented':
            chord = [root, root + 4, root + 8]
        elif chord_type == 'sus2':
            chord = [root, root + 2, root + 7]
        elif chord_type == 'sus4':
            chord = [root, root + 5, root + 7]
        chords.append(chord)
    return chords

In [28]:
#a total of 13 types of chords (7 basic chords + 6 extended chords)

start_note = 36  # C2
end_note = 96  # C7

# Generate all major chords (triads, sevenths, ninths)
major_triads = generate_chords(start_note, end_note, chord_type='major',num_notes=3)
major_sevenths = generate_chords(start_note, end_note, chord_type='major',num_notes=4)
major_ninths = generate_chords(start_note, end_note, chord_type='major',num_notes=5)

#generate all minor chords (triads, sevenths, ninths)
minor_triads = generate_chords(start_note, end_note, chord_type='minor',num_notes=3)
minor_sevenths = generate_chords(start_note, end_note, chord_type='minor',num_notes=4)
minor_ninths = generate_chords(start_note, end_note, chord_type='minor',num_notes=5)

#gene rate all dominant 7th chords (sevenths, ninths)
dom7_sevenths = generate_chords(start_note, end_note, chord_type='dom7',num_notes=4)
dom7_ninths = generate_chords(start_note, end_note, chord_type='dom7',num_notes=5)

#generate all diminished chords (triads)
dim_triads = generate_chords(start_note, end_note, chord_type='diminished',num_notes=3)

#generate all augmented chords (triads)
aug_triads = generate_chords(start_note, end_note, chord_type='augmented',num_notes=3)

#generate all sus2 chords (triads)
sus2_triads = generate_chords(start_note, end_note, chord_type='sus2',num_notes=3)

#generate all sus4 chords (triads)
sus4_triads = generate_chords(start_note, end_note, chord_type='sus4',num_notes=3)

In [34]:
#Get the sf2 file from here https://drive.google.com/file/d/1UpggAFKqgrcjWweZJNAQspuT9zo-hotJ/view

fs = FluidSynth('Piano Collection-SF4U-v1.5.sf2') #change this to the path of the sf2 file

def play_chord(chord, chord_name, directory='.'):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    for i, chord in enumerate(chord):
        # Create a new MIDI file
        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        # Change to a different instrument (0 = Acoustic Grand Piano)
        track.append(Message('program_change', program=0, time=0))

        # Add the notes of the chord
        for note in chord:
            track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=1920))
        for note in chord:
            track.append(Message('note_off', note=note, velocity=64, time=0))

        # Save the MIDI file
        midi_filename = os.path.join(directory, f'{chord_name}{i + 36:02d}.mid')
        mid.save(midi_filename)

        # Convert the MIDI file to a WAV file
        wav_filename = os.path.join(directory, f'{chord_name}{i + 36:02d}.wav')
        fs.midi_to_audio(midi_filename, wav_filename)

        # Clean up the MIDI file
        os.remove(midi_filename)

In [ ]:
#make a directory to store the all the chords. I organized them by chord type in a single folder named "chords"
# Major chords
play_chord(major_triads, 'major_triad', directory='directory/major_triad')
play_chord(major_sevenths, 'major_seventh', directory='directory/major_seventh')
play_chord(major_ninths, 'major_ninth', directory='/directory/major_ninth')



In [ ]:
#Minor chords
play_chord(minor_triads, 'minor_triad', directory='/chords/minor_triad')
play_chord(minor_sevenths, 'minor_seventh', directory='/chords/minor_seventh')
play_chord(minor_ninths, 'minor_ninth', directory='/chords/minor_ninth')

In [ ]:
#Dominant 7th chords
play_chord(dom7_sevenths, 'dom7_seventh', directory='/chords/dom7_seventh')
play_chord(dom7_ninths, 'dom7_ninth', directory='/chords/dom7_ninth')

In [ ]:
#Diminished chords
play_chord(dim_triads, 'diminished', directory='/chords/diminished')

#Augmented chords
play_chord(aug_triads, 'augmented', directory='/chords/augmented')

#Sus2 chords
play_chord(sus2_triads, 'sus2', directory='/chords/sus2')

#Sus4 chords
play_chord(sus4_triads, 'sus4', directory='/chords/sus4')